In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import gc
import io
import os

from IPython.display import display
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 200)
pd.reset_option('display.float_format')
pd.set_option('display.max_colwidth', None)

from sitecustomize import ROOT # lib này được khởi tạo ban đầu dự án

import helpers.view as view
import helpers.EDA as EDA
import helpers.config as config
import modules.utils as utils

importlib.reload(view)
importlib.reload(EDA)
importlib.reload(utils)
importlib.reload(config)

use_cols = config.use_cols
prev_use_cols = config.prev_use_cols

In [58]:
def cache_clear():
    for var in list(globals()):  
        if var not in _keep_vars and not var.startswith("_"):  
            del globals()[var]  

    gc.collect()
_keep_vars = set(globals().keys())  # lưu biến gốc


# installments payemnts

In [59]:
installments = pd.read_pickle(ROOT + "/data/pkl/installments_payments.p")

In [60]:
prev = pd.read_pickle(ROOT + "/data/pkl/previous_application.p")[prev_use_cols]

In [61]:
train = pd.read_pickle(ROOT + "/data/pkl/application_train.p")[use_cols]

In [62]:
test = pd.read_pickle(ROOT + "/data/pkl/application_test.p")[use_cols]

In [63]:
installments.sort_values(["SK_ID_PREV", "DAYS_ENTRY_PAYMENT"], ascending=[True, True], inplace=True)
installments.reset_index(drop=True, inplace=True)

In [64]:
installments["index"] = installments.index

In [65]:
prev["exist"] = 1
merged_df = pd.merge(
    installments[installments["SK_ID_PREV"].isin(installments[(installments["AMT_INSTALMENT"]==0) | (installments["AMT_INSTALMENT"].isnull())]["SK_ID_PREV"].unique())], 
    prev,
    on="SK_ID_PREV", 
    how="left"
)

In [66]:
### fill nan

median_instalment = merged_df.groupby("SK_ID_PREV")["AMT_INSTALMENT"].transform("median")

payment_sum = merged_df.groupby(["SK_ID_PREV", "NUM_INSTALMENT_NUMBER"])["AMT_PAYMENT"].sum() # vì có nhiều lần trả cho một INSTALMENT nên lấy sum
median_payment = merged_df["SK_ID_PREV"].map(payment_sum.groupby("SK_ID_PREV").median()) 
median_annuity = pd.Series(
    np.where(
        (median_instalment == 0) | median_instalment.isna(),   # annuity = null => lấy trung vị instalment => instalment = null lấy trung vị payment
        median_payment,  
        median_instalment  
    ),
    index=merged_df.index
)
merged_df["AMT_ANNUITY"] = merged_df["AMT_ANNUITY"].fillna(median_annuity)

mask = (merged_df["NUM_INSTALMENT_VERSION"] != 0) & (merged_df["AMT_PAYMENT"] > 0) # chỉ lấy những bản ghi có PAYMENT
merged_df.loc[mask, "AMT_INSTALMENT"] = merged_df.loc[mask, "AMT_INSTALMENT"].replace(0, np.nan)
merged_df["AMT_INSTALMENT"] = merged_df["AMT_INSTALMENT"].fillna(merged_df["AMT_ANNUITY"])

In [67]:
merged_df = merged_df.rename(columns={"AMT_INSTALMENT": "filled_AMT_INSTALMENT"}) # đổi tên cột và index thành cột
merged_df[merged_df["NUM_INSTALMENT_VERSION"]!=0][["index", "SK_ID_PREV", "filled_AMT_INSTALMENT"]]

,index,SK_ID_PREV,filled_AMT_INSTALMENT
0,265051,1030407,7616.655
1,265052,1030407,7616.655
2,265053,1030407,7616.655
3,265054,1030407,7616.655
4,265055,1030407,7616.655
...,...,...,...
1160,13269555,2797890,3356.865
1161,13269556,2797890,3356.865
1162,13269557,2797890,3356.865
1163,13269558,2797890,3356.865


In [68]:
installments = installments.merge(
    merged_df[merged_df["NUM_INSTALMENT_VERSION"]!=0][["index", "SK_ID_PREV", "filled_AMT_INSTALMENT"]],
    on=["index", "SK_ID_PREV"],  
    how="left"
)

In [69]:
installments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,index,filled_AMT_INSTALMENT
0,1000001,158271,1.0,1,-268,-294.0,6404.310,6404.310,0,NaN
1,1000001,158271,2.0,2,-238,-244.0,62039.115,62039.115,1,NaN
2,1000002,101962,1.0,1,-1600,-1611.0,6264.000,6264.000,2,NaN
3,1000002,101962,1.0,2,-1570,-1575.0,6264.000,6264.000,3,NaN
4,1000002,101962,1.0,3,-1540,-1559.0,6264.000,6264.000,4,NaN
...,...,...,...,...,...,...,...,...,...,...
13605396,2843499,314148,1.0,6,-1053,-1074.0,16074.000,16074.000,13605396,NaN
13605397,2843499,314148,1.0,7,-1023,-1047.0,16074.000,16074.000,13605397,NaN
13605398,2843499,314148,1.0,8,-993,-1018.0,16074.000,16074.000,13605398,NaN
13605399,2843499,314148,1.0,9,-963,-980.0,16074.000,16074.000,13605399,NaN


In [70]:
installments.loc[~installments["filled_AMT_INSTALMENT"].isnull(), "AMT_INSTALMENT"] = installments["filled_AMT_INSTALMENT"]

In [71]:
installments = installments.drop(["filled_AMT_INSTALMENT"], axis=1)

# feature engineering

In [72]:
installments["days_delayed_payment"] = installments["DAYS_ENTRY_PAYMENT"] - installments["DAYS_INSTALMENT"]

installments["AMT_PAYMENT-s-AMT_INSTALMENT"] = installments["AMT_PAYMENT"] - installments["AMT_INSTALMENT"] # chênh lệch giữa số tiền trả thực tế và số tiền phải trả theo quy định
installments["AMT_PAYMENT-d-AMT_INSTALMENT"] = installments["AMT_PAYMENT"] / installments["AMT_INSTALMENT"] # tỉ lệ giữa số tiền trả thực tế và số tiền phải trả theo quy định

installments["days_weighted_delayed_payment"] = installments["days_delayed_payment"] * installments["AMT_PAYMENT-d-AMT_INSTALMENT"] # delay trả góp * tỉ lệ trả đủ (trọng số)
installments["days_weighted_delay_tsw3"] = installments['days_weighted_delayed_payment'] * (1 + (installments['DAYS_ENTRY_PAYMENT'] * 0.0003)) # time series weight decay = 0.0003

# Days past due and days before due (không lấy giá trị âm)
installments['DPD'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']
installments['DBD'] = installments['DAYS_INSTALMENT'] - installments['DAYS_ENTRY_PAYMENT']
installments['DPD'] = installments['DPD'].apply(lambda x: x if x > 0 else 0)
installments['DBD'] = installments['DBD'].apply(lambda x: x if x > 0 else 0)

installments['month'] = (installments['DAYS_ENTRY_PAYMENT']/30).map(np.floor)

In [73]:
prev['CNT_PAYMENT'].replace(0, np.nan, inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_24076\1137191341.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  prev['CNT_PAYMENT'].replace(0, np.nan, inplace=True)


In [74]:
installments = installments.merge(prev[["SK_ID_PREV", "CNT_PAYMENT", "AMT_ANNUITY"]], on="SK_ID_PREV", how='left')

In [75]:
# prev

installments["NUM_INSTALMENT_ratio"] = installments["NUM_INSTALMENT_NUMBER"] / installments["CNT_PAYMENT"] 
installments['AMT_PAYMENT-d-AMT_ANNUITY'] = installments['AMT_PAYMENT'] / installments['AMT_ANNUITY']

In [76]:
# train test
trte = utils.get_trte(train, test)

In [77]:
drop_cols = list(set(list(trte.columns) + ["CNT_PAYMENT", "AMT_ANNUITY", "index"]))
for col in ["SK_ID_PREV", "SK_ID_CURR"]:
    if col in drop_cols:
        drop_cols.remove(col)

In [78]:
installments = installments.merge(trte, on="SK_ID_CURR", how='left')

In [79]:
installments['DAYS_ENTRY_PAYMENT-s-app_DAYS_BIRTH']             = installments['DAYS_ENTRY_PAYMENT'] - installments['app_DAYS_BIRTH']
installments['DAYS_ENTRY_PAYMENT-s-app_DAYS_EMPLOYED']          = installments['DAYS_ENTRY_PAYMENT'] - installments['app_DAYS_EMPLOYED']
installments['DAYS_ENTRY_PAYMENT-s-app_DAYS_REGISTRATION']      = installments['DAYS_ENTRY_PAYMENT'] - installments['app_DAYS_REGISTRATION']
installments['DAYS_ENTRY_PAYMENT-s-app_DAYS_ID_PUBLISH']        = installments['DAYS_ENTRY_PAYMENT'] - installments['app_DAYS_ID_PUBLISH']
installments['DAYS_ENTRY_PAYMENT-s-app_DAYS_LAST_PHONE_CHANGE'] = installments['DAYS_ENTRY_PAYMENT'] - installments['app_DAYS_LAST_PHONE_CHANGE']

installments['AMT_PAYMENT-d-app_AMT_INCOME_TOTAL'] = installments['AMT_PAYMENT'] / installments['app_AMT_INCOME_TOTAL']
installments['AMT_PAYMENT-d-app_AMT_CREDIT']      = installments['AMT_PAYMENT'] / installments['app_AMT_CREDIT']
installments['AMT_PAYMENT-d-app_AMT_ANNUITY']     = installments['AMT_PAYMENT'] / installments['app_AMT_ANNUITY']
installments['AMT_PAYMENT-d-app_AMT_GOODS_PRICE'] = installments['AMT_PAYMENT'] / installments['app_AMT_GOODS_PRICE']

In [80]:
installments.replace(np.inf, np.nan, inplace=True)
installments.replace(-np.inf, np.nan, inplace=True)

In [81]:
installments.drop(columns=drop_cols, inplace=True)

In [83]:
installments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,days_delayed_payment,AMT_PAYMENT-s-AMT_INSTALMENT,AMT_PAYMENT-d-AMT_INSTALMENT,days_weighted_delayed_payment,days_weighted_delay_tsw3,DPD,DBD,month,NUM_INSTALMENT_ratio,AMT_PAYMENT-d-AMT_ANNUITY,DAYS_ENTRY_PAYMENT-s-app_DAYS_BIRTH,DAYS_ENTRY_PAYMENT-s-app_DAYS_EMPLOYED,DAYS_ENTRY_PAYMENT-s-app_DAYS_REGISTRATION,DAYS_ENTRY_PAYMENT-s-app_DAYS_ID_PUBLISH,DAYS_ENTRY_PAYMENT-s-app_DAYS_LAST_PHONE_CHANGE,AMT_PAYMENT-d-app_AMT_INCOME_TOTAL,AMT_PAYMENT-d-app_AMT_CREDIT,AMT_PAYMENT-d-app_AMT_ANNUITY,AMT_PAYMENT-d-app_AMT_GOODS_PRICE
0,1000001,158271,1.0,1,-268,-294.0,6404.310,6404.310,-26.0,0.0,1.0,-26.0,-23.7068,0.0,26.0,-10.0,0.083333,1.000000,19133.0,1390.0,364.0,707.0,5.0,0.071159,0.031431,0.294471,0.035580
1,1000001,158271,2.0,2,-238,-244.0,62039.115,62039.115,-6.0,0.0,1.0,-6.0,-5.5608,0.0,6.0,-9.0,0.166667,9.687088,19183.0,1440.0,414.0,757.0,55.0,0.689323,0.304472,2.852570,0.344662
2,1000002,101962,1.0,1,-1600,-1611.0,6264.000,6264.000,-11.0,0.0,1.0,-11.0,-5.6837,0.0,11.0,-54.0,0.166667,1.000000,9626.0,585.0,6459.0,1877.0,621.0,0.030933,0.055680,0.530286,0.055680
3,1000002,101962,1.0,2,-1570,-1575.0,6264.000,6264.000,-5.0,0.0,1.0,-5.0,-2.6375,0.0,5.0,-53.0,0.333333,1.000000,9662.0,621.0,6495.0,1913.0,657.0,0.030933,0.055680,0.530286,0.055680
4,1000002,101962,1.0,3,-1540,-1559.0,6264.000,6264.000,-19.0,0.0,1.0,-19.0,-10.1137,0.0,19.0,-52.0,0.500000,1.000000,9678.0,637.0,6511.0,1929.0,673.0,0.030933,0.055680,0.530286,0.055680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13605396,2843499,314148,1.0,6,-1053,-1074.0,16074.000,16074.000,-21.0,0.0,1.0,-21.0,-14.2338,0.0,21.0,-36.0,0.100000,1.000000,21494.0,-366317.0,11960.0,1453.0,291.0,0.238133,0.038752,0.907982,0.051029
13605397,2843499,314148,1.0,7,-1023,-1047.0,16074.000,16074.000,-24.0,0.0,1.0,-24.0,-16.4616,0.0,24.0,-35.0,0.116667,1.000000,21521.0,-366290.0,11987.0,1480.0,318.0,0.238133,0.038752,0.907982,0.051029
13605398,2843499,314148,1.0,8,-993,-1018.0,16074.000,16074.000,-25.0,0.0,1.0,-25.0,-17.3650,0.0,25.0,-34.0,0.133333,1.000000,21550.0,-366261.0,12016.0,1509.0,347.0,0.238133,0.038752,0.907982,0.051029
13605399,2843499,314148,1.0,9,-963,-980.0,16074.000,16074.000,-17.0,0.0,1.0,-17.0,-12.0020,0.0,17.0,-33.0,0.150000,1.000000,21588.0,-366223.0,12054.0,1547.0,385.0,0.238133,0.038752,0.907982,0.051029


In [84]:
installments.to_pickle(ROOT + "/data/processed/f201_installments_payments.p")

In [85]:
installments[installments['days_delayed_payment']>0].to_pickle(ROOT + "/data/processed/f201_installments_payments_delay.p")
installments[installments['days_delayed_payment']<=0].to_pickle(ROOT + "/data/processed/f201_installments_payments_notdelay.p")

In [86]:
_keep_vars.update(["installments"])
cache_clear()